# Herval Deep Mailing

In [ ]:
import pandas as pd
import dateutil.parser as parser
import os.path
import math
import logging
import numpy as np
from multiprocessing import Pool
from datetime import datetime
import gc

In [ ]:
log_location = "../logs/"
arquivo_chamadas = "../data/inputs/Herval.PRIMEIRA.BASE.xls"

pd.options.display.max_columns = 150
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
logger = logging.getLogger()
logger.handlers = []
logger = logging.getLogger(__name__)
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'prepare_data.log.' + \
                                          datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

In [ ]:
def limpar_df(pagantes):
    del pagantes['CPF']
    del pagantes['CPFINT']
    del pagantes['STATUS']
    del pagantes['NOME_MAE']
    del pagantes['NOME_PAI']
    del pagantes['NACIONALIDADE']
    del pagantes['DATA_NASCIMENTO']
    del pagantes['CPF_CONJUGE']
    del pagantes['NOME_CONJUGE']
    del pagantes['CARGO_CONJUGE']
    del pagantes['RELACIONAMENTO']
    del pagantes['TELEFONE']
    del pagantes['REFERENCIA_PESSOAL_2']
    del pagantes['RELACIONAMENTO2']
    del pagantes['TELEFONE2']
    del pagantes['DATA_DE_NASCIMENTO1']
    del pagantes['DATA_NASCIMENTO_CONJUGE']
    del pagantes['CARGO']
    del pagantes['PROFISSAO']
    del pagantes['ENDERECO_BAIRRO']
    del pagantes['ENDERECO_CIDADE']
    del pagantes['ENDERECO_COMPLEMENTO']
    del pagantes['ENDERECO_NUMERO']
    del pagantes['ENDERECO_STATUS']
    del pagantes['ENDERECO_UF']
    del pagantes['EMAIL']
    del pagantes['PROCESSOS_JUDICIAIS']
    del pagantes['PROCESSOS_ATIVO']
    del pagantes['PROCESSOS_PASSIVO']
    del pagantes['PROCESSOS_OUTRA_PARTE']
    del pagantes['MUNICIPIO_VOTACAO']
    del pagantes['UF_VOTACAO']
    del pagantes['ENDERECO_COMERCIAL_LOGRADOURO']
    del pagantes['ENDERECO_COMERCIAL_NUMERO']
    del pagantes['ENDERECO_COMERCIAL_COMPLEMENTO']
    del pagantes['ENDERECO_COMERCIAL_BAIRRO']
    del pagantes['ENDERECO_COMERCIAL_MUNICIPIO']
    del pagantes['ENDERECO_COMERCIAL_UF']
    del pagantes['ENDERECO_RESIDENCIAL_LOGRADOURO']
    del pagantes['ENDERECO_RESIDENCIAL_NUMERO']
    del pagantes['ENDERECO_RESIDENCIAL_COMPLEMENTO']
    del pagantes['ENDERECO_RESIDENCIAL_BAIRRO']
    del pagantes['ENDERECO_RESIDENCIAL_MUNICIPIO']
    del pagantes['ENDERECO_RESIDENCIAL_UF']
    del pagantes['SEGURO_DESEMPREGO_HISTORICO']
    del pagantes['SEGURO_DESEMPREGO_DATA_PARCELA']
    return pagantes

In [ ]:
def func_str(x):
    return str(x).replace(" ","_").upper().strip()

def func_strip(x):
    return str(x).strip()

def func_start_ALTA(x):
    return str(x).startswith('ALTA')

In [ ]:
def ConverterPAGOU(x):
    return "1" if x == 1 else "0"
def ConverterSimNao(x):
    return "1" if x == "Sim" else "0"

In [ ]:
converters = { 
    "PAGOU" : ConverterPAGOU,
    "SITUACAO_CPF": func_str,
    "SEXO": func_str,
    "ESTABILIDADE_EMPREGO": func_str,
    "RESTITUICAO_2016": ConverterSimNao,
    "EMPRESAS_BEM_CAPITAL_PROCESSO_CONTRA": ConverterSimNao,
    "FALECIMENTO": ConverterSimNao,
    "SERVIDOR_PUBLICO": ConverterSimNao,
    "VINCULO_CLT": ConverterSimNao,
    "BOLSA_FAMILIA": ConverterSimNao,
    "PGFN": ConverterSimNao,
    "VEICULOS": ConverterSimNao,
    "IMOVEIS": ConverterSimNao,
    "EMPRESA": ConverterSimNao
}

df_dtypes= {

}

In [ ]:
pagantes = pd.read_excel(arquivo_chamadas, dtype=df_dtypes, converters = converters)
logging.debug("CSV carregado, limpando colunas desnecessarias")
pagantes = limpar_df(pagantes)


In [ ]:
for coluna in pagantes.head(10).columns.values:
    print(coluna)           

In [ ]:
print(pagantes.head(10))

In [ ]:
for coluna in pagantes.head(10).columns.values:
    print('coluna:' + coluna, pagantes[''+coluna].unique())

In [ ]:
def CreateColumnStr(cols, df, source_col):
    for col in cols:
        df['NORM_' + source_col + "_" + str(col)] = df.apply(lambda row: 1 if func_str(row[source_col]) == func_str(col) else 0, axis=1)
    return df

def RemoveOutliers(df, source_col, min, max):
    df['NORM_' + source_col] = df.apply(lambda row: 0   if row[source_col] < min else row[source_col] , axis=1)
    df['NORM_' + source_col] = df.apply(lambda row: max if row[source_col] > max else row[source_col] , axis=1)
    return df

def CreateProportion(df, source_col):
    max = df[source_col].mean() * 8
    min = df[source_col].min()
    print("({}) - {}, {}, {}, {}".format(source_col, max,min, df[source_col].mean()))
    
    df = RemoveOutliers(df, source_col, min, max)
    
    max = df['NORM_' + source_col].max()
    min = df['NORM_' + source_col].min()
    print("({}) - {}, {}".format(source_col, max,min))
    
    df['NORM_' + source_col] = df.apply(lambda row: 0 if row['NORM_' + source_col] == 0 else ((row['NORM_' + source_col] - min) / max) , axis=1)
    return df

In [ ]:
pagantes = CreateColumnStr(pagantes['PAGOU'].unique(), pagantes, 'PAGOU')
pagantes = CreateColumnStr(pagantes['SITUACAO_CPF'].unique(), pagantes, 'SITUACAO_CPF')
pagantes = CreateColumnStr(pagantes['FALECIMENTO'].unique(), pagantes, 'FALECIMENTO')
pagantes = CreateColumnStr(pagantes['SEXO'].unique(), pagantes, 'SEXO')
pagantes = CreateColumnStr(pagantes['ESTABILIDADE_EMPREGO'].unique(), pagantes, 'ESTABILIDADE_EMPREGO')
pagantes = CreateColumnStr(pagantes['SERVIDOR_PUBLICO'].unique(), pagantes, 'SERVIDOR_PUBLICO')
pagantes = CreateColumnStr(pagantes['VINCULO_CLT'].unique(), pagantes, 'VINCULO_CLT')
pagantes = CreateColumnStr(pagantes['BOLSA_FAMILIA'].unique(), pagantes, 'BOLSA_FAMILIA')
pagantes = CreateColumnStr(pagantes['PGFN'].unique(), pagantes, 'PGFN')
pagantes = CreateColumnStr(pagantes['VEICULOS'].unique(), pagantes, 'VEICULOS')
pagantes = CreateColumnStr(pagantes['IMOVEIS'].unique(), pagantes, 'IMOVEIS')
pagantes = CreateColumnStr(pagantes['EMPRESA'].unique(), pagantes, 'EMPRESA')
pagantes = CreateColumnStr(pagantes['RESTITUICAO_2016'].unique(), pagantes, 'RESTITUICAO_2016')
pagantes = CreateColumnStr(pagantes['EMPRESAS_BEM_CAPITAL_PROCESSO_CONTRA'].unique(), pagantes, 'EMPRESAS_BEM_CAPITAL_PROCESSO_CONTRA')

pagantes = CreateProportion(pagantes, 'VALOR_DIVIDA')
pagantes = CreateProportion(pagantes, 'RENDA_MENSAL_CONJUGE')
pagantes = CreateProportion(pagantes, 'SALARIO')
pagantes = CreateProportion(pagantes, 'CONTRATO_ATRASO')
pagantes = CreateProportion(pagantes, 'IDADE')
pagantes = CreateProportion(pagantes, 'PROCESSOS_TOTAL')


pagantes.head(100)